In [12]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import basis, fidelity, sigmax, sigmay, sigmaz, tensor, about
from qutip_qip.circuit import QubitCircuit, Gate
from qutip_qip.compiler import GateCompiler, Instruction, SpinChainCompiler
from qutip_qip.device import Model, ModelProcessor
from qutip_qip.noise import Noise
import qutip_qip

## Customizing the Hamiltonian model

Want sigma x and sigma y control

In [36]:
class MyModel(Model):
    """A custom Hamiltonian model with sigmax and sigmay control."""

    def get_control(self, label):
        """
        Get an available control Hamiltonian.
        For instance, sigmax control on the zeroth
        qubits is labeled "sx0".

        Args:
            label (str): The label of the Hamiltonian

        Returns:
            The Hamiltonian and target qubits as a tuple
            (qutip.Qobj, list).
        """
        targets = int(label[2:])
        if label[:2] == "sx":
            return 2 * np.pi * sigmax() / 2, [targets]
        elif label[:2] == "sy":
            return 2 * np.pi * sigmay() / 2, [targets]
        else:
            raise NotImplementedError("Unknown control.")


In [37]:
num_qubits = 2
processor = ModelProcessor(model=MyModel(num_qubits))

In [38]:
circuit = QubitCircuit(num_qubits)
circuit.add_gate("X", targets=1)
circuit.add_gate("X", targets=0)
circuit

In [39]:
initTime = 0
finalTime = 100e-9 # 100ns
samplingRate = 5898.24e6 # 5.89824 GS/s

VStd = 0.01
phaseStd = np.sqrt(2) / 180 * np.pi
detuningStd = 52769

omega = 20e6 # 20MHz
VNaught = 1
hbar = 1
phiNaught = 0
aNaught = 25 / 46

def operator(t, y):
    V = VNaught + np.random.normal(scale=VStd)
    phi = phiNaught + np.random.normal(scale=phaseStd)
    I = np.cos(phi)
    Q = np.sin(phi)

    dwt = np.random.normal(scale=detuningStd) * t
    s = aNaught - (1 - aNaught) * np.cos(2 * np.pi * t / finalTime)
    H = omega * np.pi * V * s * ((-I * np.cos(dwt) + Q * np.sin(dwt)) * sigmaX + (I * np.sin(dwt) - Q * np.cos(dwt)) * sigmaY)
    return H/(1j* hbar) * y

In [41]:
def mygate_compiler(gate, args):
    targets = gate.targets  # target qubit

    theta = gate.arg_value
    coeff1 = np.concatenate([np.linspace(0, 10, 50),
                             np.linspace(10, 0, 50), [0]]) / 50
    coeff2 = np.concatenate([np.linspace(0, 10, 50),
                             np.linspace(10, 0, 50), [0]]) / 50
    #  save the information in a tuple (pulse_name, coeff)
    pulse_info = [
        ("sx" + str(targets[0]), theta * coeff1),
        ("sx" + str(targets[1]), theta * coeff2),
    ]
    tlist = np.linspace(0, 1, len(coeff1))
    return [Instruction(gate, tlist, pulse_info)]

In [42]:
processor = ModelProcessor(model=MyModel(num_qubits, h_x=1.0, h_z=1.0, g=0.1))
processor.native_gates = ["ISWAP", "RX", "RZ"]

# processor.num_qubits, processor.params
# access directly the information in the model.
compiler = mygate_compiler(processor.num_qubits, processor.params)
processor.load_circuit(circuit, compiler=compiler)
result = processor.run_state(init_state=basis([2, 2], [0, 0]))
result.states[-1]

AttributeError: 'int' object has no attribute 'targets'